# Hands-on: Exploratory Data Analysis (EDA)

## Overview

In this hands-on activity, you will conduct Exploratory Data Analysis (EDA) on your cleansed dataset.

You will learn about:
1. Checking correlations: Identifying features highly correlated with reservation cancellations.
2. Performing descriptive analysis to gain a deeper understanding of underlying situations and issues.
3. Summarizing insights and drawing inferences or conclusions based on the analysis results.

## Setup

In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

## Load Data
- Replace this part with your own code. Copy from 'Data Preparation' notebook and change the input CSV file name to 'hotel_bookings_v1.csv'

In [ ]:
# Replace this part with your own code
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PjXGOLvd9BTXHT3f_wi2ujiwywR5hnfK7tAJkfmahpxu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'mlpredictivemodel-donotdelete-pr-se3ulnjuojrkgg'
object_key = 'hotel_bookings_v1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head(10)

## 1. Check Correlation between Columns (Continuous Variables)

In [ ]:
# Compute correlation
df_corr = df.drop('agent', axis=1) # 'agent' is ID, not numerically meaningful

In [ ]:
# Plot the heatmap to see correlation with columns
fig, ax = plt.subplots(figsize=(20,18))
#fig, ax = plt.subplots() #without explicitly defining size
plt.tight_layout()
sns.heatmap(df_corr.corr(), annot=True, ax=ax);

In [ ]:
# Save heatmap as image
#The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform API
import io
from project_lib import Project

project = Project(None, '<my_project_id>', '<my_project_token>')
pc = project.project_context

# Showing Project, Bucket and Assets
print('Project Name: {0}'.format(project.get_name()))
print('Project Description: {0}'.format(project.get_description()))
print('Project Bucket Name: {0}'.format(project.get_project_bucket_name()))
print('Project Assets (Connections): {0}'.format(project.get_assets(asset_type='connection')))

# Save Matplotlib figure to a bytes buffer
buf = io.BytesIO()
fig.savefig(buf, format='png', bbox_inches='tight', dpi=300)
buf.seek(0)  # Reset the buffer position to the beginning

# Convert bytes buffer to bytes
fig_bytes = buf.read()

# Save bytes to your project storage
project.save_data(data=fig_bytes, file_name='heatmap.png', overwrite=True)

In [ ]:
# Calculate correlation matrix
corr_matrix = df_corr.corr()

# Unstack correlation matrix
corr_values = corr_matrix.unstack()

# Create a DataFrame from the unstacked correlation matrix
corr_df = pd.DataFrame(corr_values, columns=['correlation'])

# Reset the index to create two columns for feature1 and feature2
corr_df = corr_df.reset_index()
corr_df = corr_df.rename(columns={'level_0':'feature1','level_1':'feature2'})

# Add absolute correlation values
corr_df['abs_correlation'] = abs(corr_df['correlation'])

# Sort the correlation values
corr_df = corr_df.sort_values(by='abs_correlation', ascending=False)
corr_df = corr_df[corr_df['feature1']!=corr_df['feature2']]
corr_df = corr_df[corr_df['feature1']=='is_canceled']
corr_df = corr_df.reset_index(drop=True)
corr_df

## List of Top 5 Features that have high correlation with 'is_canceled'

In [ ]:
# Feature with corr value more than 0.1
corr_df[corr_df['abs_correlation']>0.1]['feature2'].to_list()

## Findings
- **Total Number of Special Requests**: There is a moderate positive correlation (0.292621) between lead time and reservation cancellations. This suggests that longer lead times may be associated with a higher likelihood of cancellations.
- **Total Number of Special Requests**: A moderate negative correlation (-0.234786) is observed between the total number of special requests and reservation cancellations. This indicates that a higher number of special requests may be linked to a lower probability of cancellations.
- **Number of Required Car Parking Spaces**: A moderate negative correlation (-0.195680) exists between the number of required car parking spaces and reservation cancellations. This implies that a greater need for parking spaces might correspond to a reduced likelihood of cancellations.
- **Booking Changes**: There is a moderate negative correlation (-0.144807) between booking changes and reservation cancellations. This suggests that alterations to bookings may lead to a decreased probability of cancellations.
- **Previous Cancellations**: A slight positive correlation (0.110170) is observed between previous cancellations and reservation cancellations. This indicates that guests with a history of cancellations may be more likely to cancel their reservations.

## 2. Exploratory Data Analysis

### 2.1 Hotel Room Occupancy

### 2.1.1 What is the occupancy (room days) over the past years?

In [ ]:
# Add a new column for the number of days (occupancy)
df['total_stays_days'] = df['total_stay_nights'] + 1

In [ ]:
# Create a new column for arrival date
df['arrival_datetime'] = pd.to_datetime(df['arrival_date_year'].astype(str)+'-'+
                                        df['arrival_date_month'].astype(str)+'-'+
                                        df['arrival_date_day_of_month'].astype(str))

In [ ]:
# Create arrival day name
df['arrival_day_name'] = df['arrival_datetime'].dt.day_name()
print(df['arrival_day_name'].unique())

In [ ]:
# Create weekday/weekend label for arrival date
df['weekday_weekend'] = df['arrival_datetime'].dt.weekday.apply(lambda x: 'weekend' if x>=5 else 'weekday')
print(df['weekday_weekend'].unique())

In [ ]:
# Create a new column called 'month_year' that contains the month-year of the arrival_datetime
df['month_year'] = df['arrival_datetime'].dt.strftime('%Y-%m')
print(df['month_year'].unique())

In [ ]:
# Count the occurrences of each unique value in the 'reservation_status' column and return a Series containing the counts.
df['reservation_status'].value_counts()

In [ ]:
# Filter occupancy data
df_checkout = df[df['reservation_status']=='Check-Out']
df_checkout.shape

In [ ]:
# Group the data by the 'month_year' column and calculate the sum of stays_in_week_days
monthly_stays = df_checkout.groupby('month_year')['total_stays_days'].sum()

# Plot
plt.figure(figsize=(13,8))
sns.lineplot(data=monthly_stays, linewidth=2.5)

# Format
plt.xlabel('Month-Year', fontsize=14)
plt.ylabel('Occupancy (Room Days)', fontsize=14)
plt.title('Occupancy (Room Days) Over the Past Years', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.show()

#### Findings
- Total monthly bookings peak in September/October.
- There is a sharp drop in total monthly bookings between September/October and January.
- We need to check for seasonality; however, more data might be required.

### 2.1.2 What is the occupancy (room days) by various features?
- Hotel Type
- Market Segment
- Customer Type
- Room Type
- Arrival Day Type (Weekday or Weekend)
- Arrival Day Name

In [ ]:
# Create multiple line plots 

# Create features to analyze occupancy trends
features = ['hotel', 'market_segment', 'customer_type', 'reserved_room_type', 'weekday_weekend', 'arrival_day_name']

for feat in features:
    # Group the data by the 'month_year' column and calculate the sum of stays_in_week_days
    monthly_stays_hotel = df_checkout.groupby(['month_year', feat])['total_stays_days'].sum().unstack()
    
    # Plot
    plt.figure(figsize=(13,8))
    sns.lineplot(data=monthly_stays_hotel, linewidth=2.5)
    
    # Format the plot
    plt.xlabel('Month-Year', fontsize=14)
    plt.ylabel('Occupancy (Room Days)', fontsize=14)
    plt.title(f'Occupancy (Room Days) by {feat}', fontsize=14)
    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)
    plt.show()

## Findings
- Occupancy rates for both hotel types are on the rise, experiencing an upward trend despite seasonal variations.
- The Online Travel Agency (T/A) segment is witnessing a surge in popularity among customers.
- There is a rapid increase in transient customer bookings.
- Room type A exhibits the highest and steadily growing demand, closely mirroring the trends observed among transient customers.
- Weekday arrivals are in higher demand compared to weekend arrivals.
- There is not a significant disparity in occupancy based on the day of arrival, indicating favorable conditions for daily check-ins and room distributions.

### 2.2 Reservation Cancellations

### 2.2.1 Total Number of Confirmed/Canceled Reservations vs. Cancellation Rate

In [ ]:
# Plot bar plot

# Compute number of confirmed and canceled reservation
confirmed = df[df['is_canceled']==0].groupby(['month_year'])['is_canceled'].size()
canceled = df[df['is_canceled']==1].groupby(['month_year'])['is_canceled'].size()

# Combine the two series into a single dataframe
df_reservations = pd.DataFrame({'Confirmed': confirmed, 'Canceled': canceled})

# Compute reservations cancellation rate
cancel_rate = (df[df['is_canceled']==1].groupby('month_year').size()) / (df.groupby('month_year').size()) * 100

# Create the figure and the axes
fig, ax1 = plt.subplots(figsize=(13,8))

# Plot the stacked barplot on the first axis
df_reservations.plot(kind='bar', stacked=True, ax=ax1)
ax1.set_xlabel('Month-Year', fontsize=14)
ax1.set_ylabel('Number of Reservations', fontsize=14)
ax1.set_title('Total Number of Confirmed/Canceled Reservations vs. Cancellation Rate', fontsize=14)
ax1.set_xticklabels(df_reservations.index, rotation=45)
ax1.legend(fontsize=14)

# Create a second y-axis with the cancellation rate
ax2 = ax1.twinx()
sns.lineplot(data=cancel_rate, linewidth=2.5, ax=ax2, color='r')
ax2.set_ylabel('Reservation Cancellation Rate (%)', fontsize=14)
ax2.set_ylim(0, max(cancel_rate) + 10)
plt.show()

#### Findings
- The percentage of canceled reservations (depicted by the orange bar) mirrors the trend of the overall number of bookings (sum of orange and blue bars).
- With the exception of the period between July 2015 and August 2015, the cancellation rate closely aligns with the pattern of reservation numbers.
- Reducing the cancellation rate is crucial for boosting sales, especially considering the indication of increasing demand.

In [ ]:
# Create scatter plot
# Compute reservation cancelations rate
cancel_rate = (df[df['is_canceled']==1].groupby('month_year').size()) / (df.groupby('month_year').size()) * 100

df_cancel_rate = pd.DataFrame(cancel_rate).reset_index()
df_cancel_rate = df_cancel_rate.rename(columns={'index': 'month_year', 0: 'cancellation_rate'})

# Plot
plt.figure(figsize=(13,8))
sns.regplot(x=df_cancel_rate.index, y='cancellation_rate', data=df_cancel_rate)

# Format
plt.xlabel('Month-Year', fontsize=14)
plt.ylabel('Reservation Cancelation Rate (%)', fontsize=14)
plt.title('Reservation Cancelations Rate (%) Over the Past Years', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#### Findings
- The monthly cancellation rate is steadily increasing over time.
- Last-minute cancellations pose a loss to the business if there are no new reservations or on-site bookings, whereas earlier cancellations may still provide ample time to accommodate new reservations from other customers.
- Cancellations resulting from changes in plans are somewhat unavoidable, but those influenced by competitive hotels or user preferences can be mitigated with appropriate strategies. 

### 2.2.2 What kind of reservations were cancelled?
- Trip type (Family (With kids)/Non Family)
- Room type
- Lead time
- Deposit type

In [ ]:
df['lead_time'].describe()

In [ ]:
df = df.assign(lead_time_level = lambda x: np.where(x['lead_time'] <= 18, 'very_low',
                                                      np.where((x['lead_time'] > 18) & (x['lead_time'] <= 69), 'low',
                                                               np.where((x['lead_time'] > 69) & (x['lead_time'] <= 160), 'high',
                                                                        np.where(x['lead_time'] > 160, 'very_high', np.nan)))))

In [ ]:
# Create a new column 'family' with a default value of 0
df['family'] = 0

# Check if the 'children' or 'babies' columns are not equal to 0 or not null
# If true, update the 'family' column to 1
df.loc[(df['children'] != 0) | (df['babies'] != 0), 'family'] = 1

In [ ]:
factors = ['hotel', 'family', 'reserved_room_type', 'lead_time_level', 'deposit_type']

for factor in factors:
    plt.rcParams['figure.figsize'] = [8, 8]
    cancel_breakdown = df[df['is_canceled']==1][factor].value_counts(normalize=True) * 100
    print('Factor: ', factor, '\n', round(cancel_breakdown,1), '\n')
    plt.pie(cancel_breakdown, labels=cancel_breakdown.index, autopct='%1.1f%%', textprops={'fontsize': 14})
    plt.title(f'Canceled Reservations by {factor}')
    plt.show()

### Findings
- City hotels experience a higher frequency of cancellations compared to resort hotels.
- Non-family trips exhibit elevated cancellation rates.
- Room type A, designed for fewer guests, demonstrates a higher cancellation rate.
- Longer lead times correlate with increased cancellation rates.
- Reservations made without pre-payment exhibit a higher cancellation rate than those with non-refundable or refundable terms.

## 3. Insights related to the relationship between Lead Time and Reservation Cancellations 

### 3.1.1 What is the difference of lead time distribution between confirmed reservations and canceled reservations?

In [ ]:
# Group data for lead_time
lead_cancel_data = df.groupby('lead_time')['is_canceled'].mean()

# Plot
plt.figure(figsize=(13, 8))
sns.regplot(x=lead_cancel_data.index, y=lead_cancel_data.values * 100)
plt.title('Effect of lead time on cancellation rate', fontsize=14)
plt.xlabel('Lead Time', fontsize=14)
plt.ylabel('Cancellation Rate (%)', fontsize=14)
plt.show()

In [ ]:
# Plot
grid = sns.FacetGrid(data=df, hue='is_canceled', height=6, xlim=(0, 600))
grid.map(sns.kdeplot, 'lead_time', fill=True)

# Format
grid.add_legend()
grid.set_axis_labels('Lead Time','Density') 

In [ ]:
# Create a new column indicating whether the reservation was confirmed or cancelled
df['status'] = df['is_canceled'].apply(lambda x: 'Cancelled' if x == 1 else 'Confirmed')

# Plot
plt.figure(figsize=(13,8))
sns.boxplot(x='status', y='lead_time', data=df)

# Format
plt.xlabel('Reservation Status', fontsize=14)
plt.ylabel('Lead Time (days)', fontsize=14)
plt.title('Comparison of Lead Time for Confirmed and Cancelled Reservations', fontsize=14)
plt.show()

### Findings
- Longer lead times tend to result in higher cancellation rates.
- Encouraging shorter lead times may decrease cancellations.
- The average lead time for cancelled reservations is double that of stayed reservations.
- Shorter lead times may discourage customers from cancelling their reservations as they are less likely to find better options later.

In [ ]:
# Compute number of confirmed/canceled reservations
bar = df.groupby(['is_canceled', 'month_year'])['lead_time'].mean().reset_index()

# Plot
plt.figure(figsize=(13,8))
sns.barplot(x=bar.month_year, y=bar.lead_time, hue=bar.is_canceled)

# Format
plt.xlabel('Month-Year', fontsize=14)
plt.ylabel('Average Lead Time', fontsize=14)
plt.title('Average Lead Time of Confirmed/Canceled Reservations', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.legend(fontsize=14)
plt.show()

### Findings
- Across all months, the average lead time for canceled reservations is consistently higher compared to confirmed reservations.
- One possible strategy to decrease the cancellation rate could be to decrease the average lead time each month.

### Canceled reservations 

In [ ]:
# Compute average lead time for canceled reservation
df[df['is_canceled']==1]['lead_time'].mean()

In [ ]:
# Filter lead time which are >144 days before arrival and show the counts of reservation by market segment
df[(df['lead_time']>144)]['market_segment'].value_counts()

### Confirmed reservations 

In [ ]:
# Compute average lead time for confirmed reservation
df[df['is_canceled']==0]['lead_time'].mean()

In [ ]:
# Filter lead time which are <80 days before arrival and show the counts of reservation by market segment
df[df['lead_time']<80]['market_segment'].value_counts()

### Last minute reservations 

In [ ]:
# Count the number of reservations made on day of arrival/ 1 day before arrival, by market segment
df[(df['lead_time']==0)|(df['lead_time']==1)]['market_segment'].value_counts()

### Findings
- Most customers who cancel are from the Online TA and group segments, due to the ease of doing so through online platforms. Group customers tend to make bulk reservations early on, giving them more flexibility to change their plans. Cancellation also in bulk.
- Customers who do not cancel are mainly from the Online TA and Offline TA/TO segments.
- Customers who book late (on the day of arrival or one day before) are mostly from the Online TA or Direct segments, due to frequently changing plans.
- To reduce cancellations, one strategy could be to offer last-minute deals and discounts to encourage Online TA customers who book early to book later with a lower lead time.

### What is trend of number of staying nights for confirmed and canceled reservations?

In [ ]:
# Compute number of confirmed/canceled reservations
confirmed = df[df['is_canceled']==0].groupby(['total_stay_nights'])['is_canceled'].size()
canceled = df[df['is_canceled']==1].groupby(['total_stay_nights'])['is_canceled'].size()

# Combine the two series into a single dataframe
df_res = pd.DataFrame({'Confirmed': confirmed, 'Canceled': canceled})

# Plot
plt.figure(figsize=(13,8))
for i, column in enumerate(df_res.columns):
    plt.bar(df_res.index, df_res[column], bottom=df_res[df_res.columns[:i]].sum(axis=1), label=column)

# Format
plt.xlabel('Number of Nights', fontsize='16')
plt.ylabel('Number of Reservations', fontsize='16')
plt.title('Total Reservations by Number of Nights', fontsize=18)
plt.legend(fontsize=14)
plt.legend(fontsize=14, bbox_to_anchor=(1, 1))
plt.xlim(-1,20)
plt.xticks(np.arange(0, 20, step=1))
plt.show()


### Findings
- Customers with reservations for 2-3 nights have a higher likelihood of canceling their bookings.

In [ ]:
# Calculate the cancellation rate by number of nights
cancel_rate = (df[df['is_canceled']==1].groupby('total_stay_nights').size()) / (df.groupby('total_stay_nights').size()) * 100

# Convert the series to a DataFrame
df_cancel_rate = pd.DataFrame(cancel_rate).reset_index().rename(columns={0: 'cancellation_rate'})

# Create a line chart to visualize the data
plt.figure(figsize=(13,8))
sns.regplot(x='total_stay_nights', y='cancellation_rate', data=df_cancel_rate)
plt.xlabel('Number of Nights')
plt.ylabel('Cancellation Rate (%)')
plt.title('Cancellation Rate (%) for Number of Nights', fontsize=18)
plt.show()

#### Findings
- The data suggests that as the length of a stay increases, the likelihood of cancellation also increases. 
- This correlation could be attributed to the fact that customers who book longer stays may have more flexibility to change their plans or it could be a result of the limited number of reservations made for longer stays, making a higher cancellation rate for these reservations more impactful in proportion.

### Whats customers nationality with highest reservations number has the highest cancellation rate?

In [ ]:
cancelled_reservations_by_country = df[df['is_canceled']==1].groupby(['country']).size()
total_reservations_by_country = df.groupby(['country']).size()

cancellation_rate = cancelled_reservations_by_country / total_reservations_by_country * 100
df_cancellation_rate = pd.DataFrame({'country': total_reservations_by_country.index, 'cancellation_rate': cancellation_rate}).reset_index(drop=True)
df_cancellation_rate = df_cancellation_rate.merge(pd.DataFrame({'country': total_reservations_by_country.index, 'total_reservation': total_reservations_by_country.values}).reset_index(drop=True), on='country', how='left')

#sort by number of reservations
df_cancellation_rate = df_cancellation_rate.sort_values(by='total_reservation', ascending=False)

#Select top 10 countries with the most reservations
top_10 = df_cancellation_rate.head(10)

plt.figure(figsize=(13,8))
sns.barplot(data=top_10, x='country', y='cancellation_rate')
plt.xlabel('Country')
plt.ylabel('Cancellation Rate (%)')
plt.title('Customers Nationality and Cancellation Rate (%)', fontsize=18)
plt.show()


#### Findings
- Portugal has the highest cancellation rate among customers, followed by Brazil, Italy, and Spain, which is more than 30%.
- To decrease the cancellation rate, it may be helpful to tailor the strategies based on specific country markets.

### 3.1.2 Who have tendency to cancel last minutes?

### Cancellation on same day or 1 day before arrival

In [ ]:
df['arrival_datetime'] = pd.to_datetime(df['arrival_datetime'])
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
df['days_difference'] = (df['arrival_datetime'] - df['reservation_status_date']).dt.days

In [ ]:
features = ['market_segment', 'customer_type', 'country']
list_dd = [0,1]

for feat in features:
    # Plot
    plt.figure(figsize=(13,8))
    sns.countplot(data=df[(df['days_difference'].isin(list_dd)) & (df['is_canceled']==1)], x=feat, 
                  order=df[(df['days_difference'].isin(list_dd)) & (df['is_canceled']==1)][feat].value_counts().index)
    
    # Format
    plt.xlabel(feat)
    plt.xticks(rotation=90)
    plt.ylabel('Number of Cancelled Reservations')
    plt.title(f'Number of Canceled Reservations and {feat}', fontsize=18)
    plt.show()

#### Findings
- Customer originated from Portugal has the highest instances of last minute cancellations.
- Given the most canceled hotel are located in Portugal (Both hotels are located in Portugal: H1 at the resort region of Algarve and H2 at the city of Lisbon), it is easier for customers originally from Portugal to cancel their reservations at the last minute.
- If these last minute cancellations can be prevented, the hotel would have ample time to fill the rooms with new guests, which will improve the occupancy.

## Summary

In this hands-on activity, you have covered the following:
1. Conduct Exploratory Data Analysis (EDA) by using different types of plots.
2. Get insights from visualizations.

These are some idea on using ML to solve identified issues:
1. Use a clustering model to categorize reservations into distinct groups based on their features, enabling us to target specific actions for each group.
    - For instance, group labeling can help us recognize reservations with similar attributes such as high lead time, frequent requests, or a history of cancellations.
    - Targeted actions, tailored for each group, can be more cost-effective and result-oriented.
    - Offer personalized promotions for groups, for instance, transient customers with a high risk of canceling can be given last-minute deals or discounts to change their behavior and prevent early bookings likely to be canceled.
    - Identify the group of reservations likely to cancel last minute and reach out to them with messages promoting other room offers or discounts.
2. Use a classification model to predict if a reservation will be canceled or not, so that we can take early action on reservations likely to be canceled.
    - For example, create a list of customers highly likely to cancel reservations.
    - Take preventive and personalized action by offering discounts or coupons or proactively asking customers if they need any special requests.
3. Use a regression model to predict the number of booking nights for new reservations for optimizing business operation or sales.  
    - Provide insight into expected revenue for new reservations.
    - Use information to optimize pricing strategies and improve sales and marketing efforts.
    - Better predict future demand, hotels can ensure adequate staffing and inventory levels to meet customer needs and maximize revenue.